In [ ]:
!pip install ratsnlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 903 kB/s 
     |████████████████████████████████| 8.0 MB 5.5 MB/s 
     |████████████████████████████████| 398 kB 74.1 MB/s 
     |████████████████████████████████| 2.8 MB 66.1 MB/s 
     |████████████████████████████████| 57 kB 4.7 MB/s 
     |████████████████████████████████| 806 kB 62.9 MB/s 
     |████████████████████████████████| 829 kB 63.2 MB/s 
     |████████████████████████████████| 140 kB 68.5 MB/s 
     |████████████████████████████████| 636 kB 60.7 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:43tcmalloc: large alloc 1147494400 bytes == 0x3a9a8000 @  0x7f1236f3e615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0

# 구글 드라이브 연동하기

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정

In [ ]:
import torch
from ratsnlp.nlpbook.qa import QATrainArguments
args = QATrainArguments(
    pretrained_model_name="bert-base-multilingual-cased",
    downstream_corpus_name="korquad-v1",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-qa",
    max_seq_length=256,
    max_query_length=32,
    doc_stride=64,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

/usr/local/lib/python3.7/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


# 랜덤 시드 고정

In [ ]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정

In [ ]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='bert-base-multilingual-cased', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-qa', max_seq_length=256, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)


# KorQuAD 1.0 다운로드


In [ ]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 47.2MB/s]
Downloading: 3.88MB [00:00, 68.1MB/s]                  


# 토크나이저 준비

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

# 학습데이터 구축
학습데이터를 만듭니다.

In [ ]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KorQuADV1Corpus()
train_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from train dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [05:54<00:00, 162.80it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽 ##고 무 ##엇 ##을 쓰 ##고 ##자 했 ##는 ##가 ? [SEP] 1839 ##년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽 ##고 그 내 ##용 ##에 마 ##음 ##이 끌 ##려 이를 소 ##재 ##로 해 ##서 하나의 교 ##향 ##곡 ##을 쓰 ##려 ##는 뜻 ##을 갖 ##는다 . 이 시 ##기 바 ##그 ##너 ##는 1838 ##년에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 [UNK] 상 ##황 ##이 ##라 좌 ##절 ##과 실 ##망 ##에 가 ##득 ##했으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만 ##나는 파 ##우스 ##트 ##의 심 ##경 ##에 공 ##감 ##했다 ##고 한다 . 또한 파 ##리에 ##서 아 ##브 ##네 ##크 ##의 지 ##휘 ##로 파 ##리 음악 ##원 관 ##현 ##악 ##단 ##이 연 ##주 ##하는 베 ##토 ##벤 ##의 교 ##향 ##곡 9 ##번 ##을 듣 ##고 깊 ##은 감 ##명을 받 ##았 ##는데 , 이 ##것이 이 ##듬 ##해 1월 ##에 파 ##우스 ##트 ##의 서 ##곡 ##으로 쓰 ##여 ##진 이 작 ##품 ##에 조 ##금 ##이 ##라 ##도 영향을 끼 ##쳤 ##으 ##리 ##라는 것은 의 ##심 ##할 여 ##지가 없다 . 여 ##기의 라 ##단 ##조 조 ##성의 경우 ##에도 그의 전 ##기에 적 ##혀

# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [ ]:
val_dataset = QADataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="val",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Creating features from val dataset file at /content/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 5533/5533 [00:35<00:00, 156.32it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 1989년 6월 30일 평 ##양 ##축 ##전에 대 ##표 ##로 파 ##견 된 인 ##물 ##은 ? [SEP] 1989년 2월 15일 여 ##의 ##도 농 ##민 폭 ##력 시 ##위를 주 ##도 ##한 혐 ##의 ( 폭 ##력 ##행 ##위 ##등 ##처 ##벌 ##에 ##관 ##한 ##법 ##률 ##위 ##반 ) 으로 지 ##명 ##수 ##배 ##되었다 . 1989년 3월 12일 서울 ##지 ##방 ##검 ##찰 ##청 공 ##안 ##부는 임 ##종 ##석 ##의 사 ##전 ##구 ##속 ##영 ##장을 발 ##부 ##받 ##았다 . 같은 해 6월 30일 평 ##양 ##축 ##전에 임 ##수 ##경 ##을 대 ##표 ##로 파 ##견 ##하여 국가 ##보 ##안 ##법 ##위 ##반 혐 ##의 ##가 추 ##가 ##되었다 . 경 ##찰 ##은 12월 18일 ~ 20일 사 ##이 서울 경 ##희 ##대학교 ##에서 임 ##종 ##석 ##이 성 ##명 발 ##표 ##를 추 ##진 ##하고 있다는 첩 ##보를 입 ##수 ##했고 , 12월 18일 오 ##전 7 ##시 40 ##분 경 가 ##스 ##총 ##과 전 ##자 ##봉 ##으로 무 ##장 ##한 특 ##공 ##조 및 대 ##공 ##과 직 ##원 12 ##명 등 22 ##명의 사 ##복 경 ##찰 ##을 승 ##용 ##차 8 ##대에 나 ##누 ##어 경 ##희 ##대학교 ##에 투 ##입 ##했다 . 1989년 12월 18일 오 ##전 8 ##시 15 ##분 경 서울 ##청 ##

# 모델 초기화

In [ ]:
from transformers import BertConfig
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

In [ ]:
from transformers import BertForQuestionAnswering
model = BertForQuestionAnswering.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

# 학습 준비

In [ ]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-qa exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


# 학습
구글 드라이브 (`/gdrive/My Drive/nlpbook/checkpoint-qa`)에 저장

In [ ]:
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 177 M 
---------------------------------------------------
177 M     Trainable params
0         Non-trainable params
177 M     Total params
709.058   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]